# Nano-porous materials

In this notebook we'll explore nano-porous materials in the molecular dynamics scheme. We'll use Lammps to simulate our systems.

In [1]:
import os
import sys

sys.path.append("..")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from read_lammps_dump import read_dump

sns.set(color_codes=True)

We start by creating a system of liquid Argon in a system of $L = 20$ unit cells with length $b = 5.72 \text{Å}$ and thermalize it at $T = 0.851$. To create the specified length, we have to convert to the correct dimensions. For $\sigma = 3.405 \text{$\mu$m}$ we do this by
\begin{align}
    b' = \frac{b}{\sigma},
\end{align}
where $b'$ is the simulation length used in the simulation. When specifying the length in the front-centered-cubic lattice Lammps uses the reduced number density $\rho'$. This is given by
\begin{align}
    \rho' = \frac{n}{{b'}^3},
\end{align}
where $n$ is the number of atoms in the specified region. For front-centered-cubic lattices this is four atoms, i.e., $n = 4$.

Having set up the system we wish to thermalize it at $T = 0.851$ (note that the temperature is already in the right dimension). We do this by using the `fix nvt`-command in Lammps, which lets us specify the temperature we wish the system to achieve. Once the temperature desired temperature is achieved we move on to creating a cylinder, or in this case, we define a cylinder region and make all atoms outside this region frozen by removing all their velocity and keeping them fixed. Thus, all the atoms inside the cylinder still "feel" the atoms outside the cylinder, but they are unable to move the fixed atoms. The cylinder we create has a radius of $R = 2 \text{nm} = 20 \text{Å}$ and is located in the center of the $yz$-plane stretching from $x = 0$ to $x = L$.

In [38]:
%%writefile scripts/liquid_argon.in
# 3d Lennard-Jones gas
units lj
dimension 3
# Periodic boundiaries
boundary p p p
atom_style atomic

variable sigma equal 3.405
variable b equal 5.72
variable reduced_density equal 4/((${b}/${sigma})^3)
variable temperature equal 0.851

# Set fcc lattice with specified density
lattice fcc ${reduced_density}
region simbox block 0 20 0 20 0 20
create_box 2 simbox
create_atoms 1 box
create_atoms 2 single -1 -1 -1

# Use a radius of 2 nm (equal to 20 Å)
variable radius equal 20/${sigma}
variable centre_y equal ly/2
variable centre_z equal lz/2
region cylinder cylinder x ${centre_y} ${centre_z} ${radius} EDGE EDGE units box

mass 1 1.0
mass 2 1.0
velocity all create ${temperature} 87287

pair_style lj/cut 3.0
pair_coeff 1 1 1.0 1.0
pair_coeff 2 2 1.0 1.0
pair_coeff 1 2 1.0 1.0

fix 1 all nvt temp ${temperature} ${temperature} 0.5
#fix 1 all nve

dump 1 all custom 10 dat/liquid_argon.lammpstrj id type x y z vx vy vz
thermo 100
# Thermalize for 3000 steps
run 300

# Group all atoms in the cylinder
group cylinder_group region cylinder
set group cylinder_group type 2
# Create another group from all the remaining atoms except for the ones in the cylinder
group frozen subtract all cylinder_group
# Set the velocity of the atoms outside the cylinder to zero
velocity frozen set 0 0 0
# Delete the "1" group
unfix 1

# Run cylinder using nve
fix 1 cylinder_group nve
run 5000

Overwriting scripts/liquid_argon.in


In [39]:
!export OMP_NUM_THREADS=4 && mpirun -np 4 lmp -in scripts/liquid_argon.in

LAMMPS (4 Jan 2019)
  using 4 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 1.67988 1.67988 1.67988
Created orthogonal box = (0 0 0) to (33.5977 33.5977 33.5977)
  1 by 2 by 2 MPI processor grid
Created 32000 atoms
  Time spent = 0.000822122 secs
Created 0 atoms
  Time spent = 5.3607e-05 secs
Neighbor list info ...
  update every 1 steps, delay 10 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 3.3
  ghost atom cutoff = 3.3
  binsize = 1.65, bins = 21 21 21
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair lj/cut, perpetual
      attributes: half, newton on
      pair build: half/bin/atomonly/newton
      stencil: half/bin/3d/newton
      bin: standard
Setting up Verlet run ...
  Unit style    : lj
  Current step  : 0
  Time step     : 0.005
Per MPI rank memory allocation (min/avg/max) = 13.76 | 13.76 | 13.76 Mbytes
Step Temp E_pair E_mol TotEng Press 
       0        0.851   -6.9322187            0   -5.6557586 

In [14]:
df, num_atoms, bounds = read_dump("liquid_argon.lammpstrj")